In [1]:
# %reload_ext autoreload
# %autoreload 2

# import os
# import sys

# # Add the root folder to the module search path
# # Get the current directory
# current_directory = os.getcwd()

# # Move two levels up (go to the parent directory of the parent directory)
# two_levels_up_directory = os.path.dirname(os.path.dirname(current_directory))

# print(two_levels_up_directory)

# sys.path.append(two_levels_up_directory)

In [2]:
import pandas as pd
from pykoi.rlhf import RLHFConfig
from pykoi.rlhf import RewardFinetuning
from pykoi.chat import RankingDatabase

/home/ubuntu/miniconda3/envs/pykoi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Train RLHF using the data from database

Let's take a look of the QA data and process it for training.

In [3]:
stack_exchange_df = pd.read_csv("ranking.csv", index_col=0)
stack_exchange_df.head()

,question,up_ranking_answer,low_ranking_answer
id,,,
1,who are u?,Unemployed,I'm a girl
2,what is the weather today?,it is rainy today,The weather today is sunny with a high of 75 ...
3,who are u?,Unemployed,I'm a girl
4,what is the weather today?,it is rainy today,The weather today is sunny with a high of 75 ...
5,how are u?,i am fine,I'm fine


In [4]:
from datasets import Dataset

print("My ranking database has {} samples".format(stack_exchange_df.shape[0]))
dataset = Dataset.from_dict(stack_exchange_df)
dataset

My ranking database has 30 samples


Dataset({
    features: ['question', 'up_ranking_answer', 'low_ranking_answer'],
    num_rows: 30
})

In [5]:
RANKING_CSV_HEADER_ID = 'id'
RANKING_CSV_HEADER_QUESTION = 'question'
RANKING_CSV_HEADER_UP_RANKING_ANSWER = 'up_ranking_answer'
RANKING_CSV_HEADER_LOW_RANKING_ANSWER = 'low_ranking_answer'
RANKING_CSV_HEADER = (
    RANKING_CSV_HEADER_ID,
    RANKING_CSV_HEADER_QUESTION,
    RANKING_CSV_HEADER_UP_RANKING_ANSWER,
    RANKING_CSV_HEADER_LOW_RANKING_ANSWER
)

In [6]:
ranking_database = RankingDatabase()
for row in stack_exchange_df.iloc[:].to_dict('records'):
    ranking_id = ranking_database.insert_ranking(question=row[RANKING_CSV_HEADER_QUESTION],
                                       up_ranking_answer=row[RANKING_CSV_HEADER_UP_RANKING_ANSWER],
                                       low_ranking_answer=row[RANKING_CSV_HEADER_LOW_RANKING_ANSWER]
                                       )
ranking_database

In [7]:
my_data_pd = ranking_database.retrieve_all_question_answers_as_pandas()
my_data_pd.head()

,ID,Question,Up Ranking Answer,Low Ranking Answer
0,1,who are u?,Unemployed,I'm a girl
1,2,what is the weather today?,it is rainy today,The weather today is sunny with a high of 75 ...
2,3,who are u?,Unemployed,I'm a girl
3,4,what is the weather today?,it is rainy today,The weather today is sunny with a high of 75 ...
4,5,how are u?,i am fine,I'm fine


In [8]:
from datasets import Dataset

my_data_pd = my_data_pd[["ID",
                        "Question",
                        "Up Ranking Answer",
                        "Low Ranking Answer"]]
print("My local database has {} samples".format(my_data_pd.shape[0]))
dataset = Dataset.from_dict(my_data_pd)
dataset

My local database has 60 samples


Dataset({
    features: ['ID', 'Question', 'Up Ranking Answer', 'Low Ranking Answer'],
    num_rows: 60
})

### Train with RLHF

In [9]:
# run supervised finetuning
config = RLHFConfig(base_model_path="databricks/dolly-v2-3b", 
                          dataset_type="local_db",
                          )
rlhf_step2_rft = RewardFinetuning(config)
rlhf_step2_rft.train_and_save("./models/rlhf_step2_rw")


Some weights of GPTNeoXForSequenceClassification were not initialized from the model checkpoint at databricks/dolly-v2-3b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Parameter 'function'=<bound method RewardFinetuning._preprocess_function of <pykoi.rlhf.rw_finetuning.RewardFinetuning object at 0x7fbb3af82950>> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


trainable params: 5,248,000 || all params: 2,651,617,280 || trainable%: 0.19791694825582068
My local database has 60 samples for RW finetuning


Map:   0%|          | 0/60 [00:00<?, ? examples/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
                                                     

Size of the train set: 54.                 Size of the validation set: 6


/home/ubuntu/miniconda3/envs/pykoi/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/ubuntu/miniconda3/envs/pykoi/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2411: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Accuracy
10,0.821100,0.495117,0.666667
